In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

2023-08-08 18:02:20.179560: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 18:02:20.666361: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 18:02:22.560979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
print(X_train_full.shape, X_test.shape)

(15480, 8) (5160, 8)


In [3]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train_full.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

2023-08-08 18:03:45.397267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-08 18:03:45.399236: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
# callback to save the model weights
save_callback = tf.keras.callbacks.ModelCheckpoint(filepath='checkpoint/', save_weights_only=True, monitor="RootMeanSquaredError", save_best_only=False)

# custom callback
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # print(logs.keys())
        if logs.get("loss") < 0.24:
            print("mean square error is less than 0.2. quiting training")
            self.model.stop_training = True



model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics="RootMeanSquaredError")

norm_layer.adapt(X_train_full)

history = model.fit(X_train_full, y_train_full, epochs=50, callbacks=[save_callback, CustomCallback()])

Epoch 1/50
484/484 [==============================] - 2s 2ms/step - loss: 0.2141 - root_mean_squared_error: 0.4627
